# Bootstrap de advertising

In [11]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import r2_score, make_scorer, roc_auc_score
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings('ignore')
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report



In [12]:
datos=pd.read_csv(r'D:\Advertising.csv')
datos.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [13]:
cols=datos.columns
new=cols.drop(["sales"])
x=datos[new]
X=sm.add_constant(x)
y=datos['sales']
modelo = sm.OLS(y, X)
results = modelo.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.895
Method:                 Least Squares   F-statistic:                     425.7
Date:                Thu, 20 Nov 2025   Prob (F-statistic):           3.94e-95
Time:                        17:41:52   Log-Likelihood:                -386.14
No. Observations:                 200   AIC:                             782.3
Df Residuals:                     195   BIC:                             798.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.0052      0.394      7.623      0.0

In [14]:
print(results.params)

const         3.005209
Unnamed: 0   -0.000580
TV            0.045776
radio         0.188383
newspaper    -0.001243
dtype: float64


In [15]:
intervalos = results.conf_int(alpha=0.05)

print(intervalos)

                   0         1
const       2.227750  3.782668
Unnamed: 0 -0.004720  0.003560
TV          0.043017  0.048535
radio       0.171328  0.205439
newspaper  -0.012942  0.010456


In [ ]:
boot_betas = []

for i in range(1000):
    sample = datos.sample(n=len(datos), replace=True)
    X_sample = sample[['Unnamed: 0','TV', 'radio', 'newspaper']]
    y_sample = sample['sales']
    X_sample = sm.add_constant(X_sample)
    model_boot = sm.OLS(y_sample, X_sample).fit()
    boot_betas.append(model_boot.params)

df_betas_boot = pd.DataFrame(boot_betas)

print(np.mean(boot_betas))
print(np.std(boot_betas))

0.6507401178154786
1.2003604831629235


In [ ]:
import optuna
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

cols=datos.columns
new=cols.drop(["sales"])
x=datos[new]
X=sm.add_constant(x)

def objective(trial):
    alpha_candidate = trial.suggest_float('alpha', 0.0001, 10.0)
    model = Ridge(alpha=alpha_candidate)
    scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    rmse = np.sqrt(-scores.mean())
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print("Mejor Alpha:", study.best_params['alpha'])

[I 2025-11-20 17:47:03,546] A new study created in memory with name: no-name-be5b3f42-bb0d-48fd-95fb-951bae750f21
[I 2025-11-20 17:47:03,586] Trial 0 finished with value: 1.7742819857413685 and parameters: {'alpha': 6.807862694629099}. Best is trial 0 with value: 1.7742819857413685.
[I 2025-11-20 17:47:03,605] Trial 1 finished with value: 1.774305319206464 and parameters: {'alpha': 0.11028046142416703}. Best is trial 0 with value: 1.7742819857413685.
[I 2025-11-20 17:47:03,624] Trial 2 finished with value: 1.7742745554801138 and parameters: {'alpha': 8.95390791181848}. Best is trial 2 with value: 1.7742745554801138.
[I 2025-11-20 17:47:03,642] Trial 3 finished with value: 1.7742932710165908 and parameters: {'alpha': 3.5607210982362347}. Best is trial 2 with value: 1.7742745554801138.
[I 2025-11-20 17:47:03,662] Trial 4 finished with value: 1.7742878308605832 and parameters: {'alpha': 5.12418478942413}. Best is trial 2 with value: 1.7742745554801138.
[I 2025-11-20 17:47:03,680] Trial 5 

Mejor Alpha: 9.9982161016562


## Default

In [16]:
df2=pd.read_csv(r"D:\Default.csv")
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   default  10000 non-null  object 
 1   student  10000 non-null  object 
 2   balance  10000 non-null  float64
 3   income   10000 non-null  float64
dtypes: float64(2), object(2)
memory usage: 312.6+ KB


In [17]:
df2['student']=df2['student'].astype(int)
df2['default']=df2['default'].astype(int)
x=df2[['student','balance','income']]
y=df2['default']
LOGR = LogisticRegression()
LOGR.fit(x, y)
print(f"Coeficiente: {LOGR.coef_}")
print(f"Intercepto: {LOGR.intercept_}")


ValueError: invalid literal for int() with base 10: 'No'

In [ ]:
# Con tus coeficientes β₀ y β₁
import numpy as np

# β₀ = intercept, β₁ = coeficiente de balance
beta_0 = LOGR.intercept_[0]
beta_1 = LOGR.coef_[0][0]

# Calcular probabilidades predichas
p = LOGR.predict_proba(X)[:, 1]  # Probabilidades de clase 1